In [ ]:
import math
import random
from math import floor
import matplotlib.pyplot as plt

fp_theta  = open('CORDIC_DAT_THETA_S0I1F16.dat' , 'w')
fp_result = open('CORDIC_DAT_RESULT_S0I1F16.dat', 'w')

FLAG_FULL_PRECISION = 0
NUMB_ITER = 16
DATA_INT_WD = 1
DATA_FRC_WD = 16
DATA_WD = DATA_INT_WD + DATA_FRC_WD

DATA_INN_ANG_WD = 19
DATA_INN_RLT_WD = 19

if FLAG_FULL_PRECISION:
    ANGLES = [math.atan(2**(-i)) for i in range(NUMB_ITER)]
else:
    ANGLES = [floor(math.atan(2**(-i)) * (2 ** DATA_INN_ANG_WD) + 0.5) / (2 ** DATA_INN_ANG_WD) for i in range(NUMB_ITER)]
for i in range(NUMB_ITER):
    print(f"{ANGLES[i] * (2 ** DATA_INN_ANG_WD)}")

K = 1
for i in range(NUMB_ITER):
    K *= math.cos(ANGLES[i])
K = floor(K * (2 ** DATA_INN_ANG_WD) + 0.5) / (2 ** DATA_INN_ANG_WD)
print(f"{K * (2 ** DATA_INN_ANG_WD)}")

def cordic_sin(theta):
    x = K
    y = 0
    z = theta
    for i in range(NUMB_ITER):
        d = 1 if z >= 0 else -1
        if FLAG_FULL_PRECISION:
            x_temp = x * (2 ** (-i))
            y_temp = y * (2 ** (-i))
        else:
            x_temp = (floor(x * (2 ** DATA_INN_RLT_WD)) >> i) / (2 ** DATA_INN_RLT_WD)
            y_temp = (floor(y * (2 ** DATA_INN_RLT_WD)) >> i) / (2 ** DATA_INN_RLT_WD)
        x_next = x - d * y_temp
        y_next = y + d * x_temp
        z_next = z - d * ANGLES[i]
        x = x_next
        y = y_next
        z = z_next
        #print(f"\n{i}:")
        #print(f"{x_temp * (2 ** DATA_INN_RLT_WD)}")
        #print(f"{y_temp * (2 ** DATA_INN_RLT_WD)}")
        #print(f"{x * (2 ** DATA_INN_RLT_WD)}")
        #print(f"{y * (2 ** DATA_INN_RLT_WD)}")
        #print(f"{z * (2 ** DATA_INN_ANG_WD)}")
        #print(f"{y * (2 ** DATA_FRC_WD)}")
    return y

errors = []
angles = range(floor(math.pi / 2 * (2 ** DATA_FRC_WD)))
for theta in angles:
    result_target = floor(math.sin(theta / (2 ** DATA_FRC_WD)) * (2 ** DATA_FRC_WD) + 0.5)
    result_calc = floor(cordic_sin(theta / (2 ** DATA_FRC_WD)) * (2 ** DATA_FRC_WD) + 0.5)
    errors.append(abs(result_calc - result_target))
    fp_theta .write("{:x}\n".format(theta                   ))
    fp_result.write("{:x}\n".format(result_calc & 0xffffffff))
    #if (result_calc != result_target):
    #    print(f"sin({theta}) = {result_target}")
    #    print(f"sin({theta}) = {result_calc}")
    #    print("ERROR")
    #    break

print(f"DONE @ ({theta})!")

#plt.rcParams['figure.dpi'] = 300
#plt.scatter(angles, errors)
#plt.xlabel('angles')
#plt.ylabel('errors')
#plt.grid(True)
#plt.show()
